In [4]:
import openai
import pandas as pd
import os
from dotenv import load_dotenv
# new
from openai import AsyncOpenAI
import base64


# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')
  
# Load the environment variables from .env file
load_dotenv()

# Retrieve the API key
openai_api_key = os.getenv('OPENAI_API_KEY')

# Configuration
openai.api_key = openai_api_key
model_name = 'gpt-4o'  # Ensure this model version supports sessions

prompt = (
"You are a classifier. In the following, I am going to provide a figure from a peer-reviewed paper, " + 
"and I want you to return a python list of 7 items, according to the following rules:" + 
"\n\n Classfier rules:" +
'1. Are there any arrows in the diagram? Respond: “Yes” or “No” \n' +
'2. If there are arrows, is there any text next to arrows (that belongs to the arrows)? Respond: “Yes”, “No”, or “Not applicable” \n' +
'3. Are there any numbered steps in the diagram? Respond: “Yes” or “No” \n' +
'4. Are there any full cycles present in the diagram? Respond: “Yes" or “No” \n' +
'5. Does the diagram show a process or flow with a single start point and a single end point? Respond: “Yes” or “No” \n' +
'6. Does the diagram contain any square components (are there any squares within the diagram)? Respond: “Yes” or “No” \n' +
'7. Does the diagram contain any circle components (are there any cicles within the diagram)? Respond: “Yes” or “No” \n'  + 
'Please return as a list of strings, with no comments or numbering.'
)

new_columns = ['image path', 'arrows', 'text with arrows', 'numbered steps', 'full cycles', 'start and end point', 'square components', 'circle components']

client = AsyncOpenAI()

for year in range(2020, 2023):

    diagram_data_path = f"../diagram_data/{year}_diagram_data.xlsx"
    diagram_data_df = pd.read_excel(f"../diagram_data/{year}_diagram_data.xlsx")
    # Load the Excel file into a DataFrame
    cropped_and_labeled_df = pd.read_excel(f"../cropped_and_labeled_image_data/{year}_cropped_and_labeled_figs.xlsx")

    count = 0
    subcategories_to_check = ["process diagram","conceptual diagram", "cycle", "hierarchical diagram", "data structure", "data display", "data collection, data analysis, data gathering diagram", "data map", "organizational chart", "timeline"]
    # Loop through rows and check for 'none selected'
    for index, row in cropped_and_labeled_df.iterrows():
        if any(row["subcategory"] == subcategory for subcategory in subcategories_to_check) and row["new image path"] not in diagram_data_df['image path'].values:
            if(row["new image path"] not in diagram_data_df["image path"].values):
                # if(count < 2):
                base64_image = encode_image("../" + row["new image path"])
                print(row["new image path"])
                image_completion = await client.chat.completions.create(model="gpt-4o", messages=[{"role": "user","content": [    
                    {
                        "type": "text", 
                        "text": prompt
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        },
                        
                    }]}])
                print(image_completion)
                response = image_completion.choices[0].message.content
                response_list = eval(response.strip())
                # Append new data to the DataFrame
                new_data = [row["new image path"]] + response_list
                new_row = pd.DataFrame([new_data], columns=diagram_data_df.columns)
                diagram_data_df = pd.concat([diagram_data_df, new_row], ignore_index=True)
                count = count + 1


    print(f'Images classified: {count}')
    # Save updated DataFrame back to Excel
    diagram_data_df.to_excel(diagram_data_path, index=False)

final_figures/2020/F1_P6_Hampel, Tracey & Weber_2020_AMJ_The Art of the Pivot.png
ChatCompletion(id='chatcmpl-9dkA3aPJpc1XrGZbUuwZuuqrMJh2M', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='["Yes", "No", "No", "No", "Yes", "Yes", "No"]', role='assistant', function_call=None, tool_calls=None), logprobs=None)], created=1719259263, model='gpt-4o-2024-05-13', object='chat.completion', system_fingerprint='fp_9cb5d38cf7', usage=CompletionUsage(completion_tokens=21, prompt_tokens=1019, total_tokens=1040))
final_figures/2020/F1_P18_Hengst et al._2020_AMJ_Toward a Process Theory of Sustainability Strategies Legitimate in Action.png
ChatCompletion(id='chatcmpl-9dkA6eQ8dFS82mw5kr8yO5FkWHUIj', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='["Yes", "Yes", "Yes", "Yes", "No", "Yes", "Yes"]', role='assistant', function_call=None, tool_calls=None), logprobs=None)], created=1719259266, model='gpt-4o-2024-05-13', object='chat.c